In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.display import display, Markdown

import sys
sys.path.insert(0, '../py_scripts')

import numpy as np
import scipy as sp
import pandas as pd
import numpy.random as rand
import numpy.linalg as la
import numpy.ma as ma
import scipy.optimize as opt
import scipy.stats as stats

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import time

import push_pull as pp
# import noise_models as noise

sns.set(context='talk', font_scale=1.0, color_codes=True, palette='deep', style='ticks', 
        rc={'mathtext.fontset': 'cm', 'xtick.direction': 'in','ytick.direction': 'in',
            'axes.linewidth': 1.5, 'figure.dpi':100, 'text.usetex':False})

In [ ]:
df = pd.read_csv("../data/old/push/u5.csv")

display(df)

df = df[(df[df.columns] > 0.0).all(axis=1)]

data = pp.PushDataSet(df['Kinase'].values, 
                      df['Substrate'].values, 
                      df['Phosphorylation'].values)



model = pp.PushAmp()

params = np.array([1.0, 2.0, 3.0])
print(model.predict(data[0], params))

df_noise = pd.read_csv("../data/old/Kinase Noise.csv")
df_noise = df_noise[(df_noise[df_noise.columns] > 0.0).all(axis=1)]

display(df_noise)

noise = pp.EmpiricalNoise(df_noise['Flag Antibody'].values, 
                          df_noise['GFP - Area'].values, 100, 100)

GFP = noise.anti_to_GFP(df['Kinase'].values, np.array([]), 42)

sns.histplot(x=GFP, y=df['Kinase'].values, log_scale=(True, True), bins=(100, 100), cbar=True)

plt.show()

# loss_func = pp.EmpiricalPlusEmpty()

# print(loss_func.loss(params, np.array([1.0, 1.0]), model, data))

# sns.heatmap(hist)

## Load Data

In [ ]:
model_dict = {'I+E': ('push', 'I+E'),
              'S+E': ('push', 'S+E'),
              'S+R': ('push', 'S+R'), 
              'RR(E) only': ('push', 'RR(E) only'), 
              'RR+A': ('push', 'RR+A')}





df_list = []
# df = pd.read_csv("../data/push_data/I+E.csv", index_col=0)
# df['construct'] = 'I+E'
# df_list.append(df)
# df = pd.read_csv("../data/push_data/S+E.csv", index_col=0)
# df['construct'] = 'S+E'
# df_list.append(df)
df = pd.read_csv("../data/push_data/S+R.csv", index_col=0)
df['construct'] = 'S+R'
df_list.append(df)
# df = pd.read_csv("../data/push_data/RR(E) only.csv", index_col=0)
# df['construct'] = 'RR(E) only'
# df_list.append(df)
# df = pd.read_csv("../data/push_data/RR+A.csv")
# df['construct'] = 'RR+A'
# df_list.append(df)


df = pd.concat(df_list)


df = df[(df[df.columns[:-1]] >= 0).all(axis=1)].rename(columns={'Kinase': 'WT_anti', 'Substrate': 'ST_anti', 'Phosphorylation': 'SpT_anti'})


df = df.sample(frac=1.0, replace=False, random_state=776)

df.reset_index(inplace=True, drop=True)

df['phospho_frac_anti'] = df['SpT_anti'] / df['ST_anti']


print(len(df.index), "/", len(df.index))

display(df)


nconstructs = df.groupby("construct").ngroups
fig, axes = plt.subplots(nconstructs, 1, figsize=(4, 4*nconstructs),
                        sharex=True, sharey=True, squeeze=False)

print(axes)

for i, (construct, group) in enumerate(df.groupby("construct")):
    
    sns.histplot(group, x='WT_anti', y='phospho_frac_anti', 
                 log_scale=(True, True), ax=axes[i, 0])
    
    axes[i, 0].hlines(1e0, xmin=1e0, xmax=1e5, color='k', linestyle='--')
    axes[i, 0].set_title(construct)

plt.show()


## Resample from Noise Models

In [ ]:
nbins_anti = 200
nbins_gfp = 200

df_writer_noise = pd.read_csv("../data/noise_data/Kinase Noise.csv")
df_writer_noise = df_writer_noise[(df_writer_noise[df_writer_noise.columns] > 0.0).all(axis=1)]

display(df_writer_noise)

writer_noise = pp.EmpiricalNoise(df_writer_noise['Flag Antibody'].values, 
                                 df_writer_noise['GFP'].values, nbins_anti, nbins_gfp)

sns.histplot(df_writer_noise, x='GFP', y='Flag Antibody', log_scale=(True, True), 
             bins=(nbins_anti, nbins_gfp), cbar=True)
plt.show()


df_substrate_noise = pd.read_csv("../data/noise_data/Substrate Noise.csv")
df_substrate_noise = df_substrate_noise[(df_substrate_noise[df_substrate_noise.columns] > 0.0).all(axis=1)]

display(df_substrate_noise)

substrate_noise = pp.EmpiricalNoise(df_substrate_noise['Myc Antibody'].values, 
                                 df_substrate_noise['GFP'].values, nbins_anti, nbins_gfp)

sns.histplot(df_substrate_noise, x='GFP', y='Myc Antibody', log_scale=(True, True), 
             bins=(nbins_anti, nbins_gfp), cbar=True)
plt.show()




for construct, group in df.groupby("construct"):
                    
    df.loc[group.index, 'WT_GFP'] = writer_noise.anti_to_GFP(group['WT_anti'].values, np.array([]), 42)
    df.loc[group.index, 'ST_GFP'] = substrate_noise.anti_to_GFP(group['ST_anti'].values, np.array([]), 42)
    
print(len(df))
df = df[(df['WT_GFP']!=-1) & (df['ST_GFP']!=-1)].reset_index(drop=True)
print(len(df))

display(df)

In [ ]:
def solve(datasets, noise_model, model_dict, param_dict, x0, bounds, verbose=False):

    if verbose:
        start = time.time()

    def func(x, args):
        
#         print(x)
                
        datasets, noise_model = args

        loss = 0.0
        norm = 0.0
        for construct in datasets:
            
            (model_type, model_id) = model_dict[construct]
            dataset = datasets[construct]
            
            N_data = dataset.size()
            
            norm += N_data
                        
            if model_type == 'push':
                
                noise_params = np.array(x)[param_dict[model_id][0]]
                model_params = 10**np.array(x)[param_dict[model_id][1]]
                
                model = pp.PushAmp()
                
#                 print(model_params, noise_params)
                
                loss += N_data * noise_model.loss(dataset, model_params, noise_params, model)                
                
            elif model_type == 'background':
                
#                 Sigma2, A, logvbgp = np.array(x)[param_dict[model_id]]
                
#                 loss += N_data * model.loss(
#                     np.array([10**logvbgp]), 
#                     np.array([Sigma2, A, 0.0]))
                pass
        
        loss /= norm
        
#         print(loss, x)
        
        return loss


    if verbose:
        print("Initial Loss:", func(x0, (datasets, noise_model)))


    res = opt.minimize(func, x0, args=((datasets, noise_model),), method='L-BFGS-B', 
                       jac='2-point', bounds=bounds, 
                       options={'iprint':101, 'eps': 1e-8, 
                                'gtol': 1e-8, 'ftol':1e-12,
                               'finite_diff_rel_step': 1e-4})
    

    if verbose:
        print("Final Loss:", res.fun)

        end = time.time()

        print("Time Elapsed", end-start, "seconds")

        print(res)
        
    
    return res


df_phospho_noise = pd.read_csv("../data/noise_data/PE Noise.csv")
df_phospho_noise = df_phospho_noise[(df_phospho_noise[df_phospho_noise.columns] > 0.0).all(axis=1)]

display(df_phospho_noise)

phospho_noise = pp.EmpiricalNoise(df_phospho_noise['PE Antibody'].values, 
                                 df_phospho_noise['GFP'].values, nbins_anti, nbins_gfp)

sns.histplot(df_phospho_noise, x='GFP', y='PE Antibody', log_scale=(True, True), 
             bins=(nbins_anti, nbins_gfp), cbar=True)
plt.show()



In [ ]:
# Only need one model of each type, so this can be recajiggered
# Next step: see what results look like to see if even in the right regime for fit params, confused why not
# is discreteness of optimization function creating issues?
# plot average antibody curve


param_dict = {}
datasets = {}

n_push = 0

param_labels = [r"$\rho$", r"$\log_{10}(v_{bg}^p)$", r"$\log_{10}(v_{WS}^p)$"]

for construct, group in df.groupby("construct"):
    
    (model_type, model_id) = model_dict[construct]
            
    if model_id not in param_dict:
        if model_type == 'background':
            param_dict[model_id] = ([0], [1])
        elif model_type == 'push':
            param_dict[model_id] = ([0], [1, 2, 3+n_push])
            n_push += 1
            
            param_labels.append(model_id + ": " + r"$\log_{10}(\alpha_{WS})$")
        
    
    if model_type == 'background':
#         model = ppamp.Background()
#         model.set_data(group['ST_GFP'].values.copy(), 
#                        group['SpT_anti'].values.copy())
        pass
    elif model_type == 'push':
        data = pp.PushDataSet(group['WT_GFP'].values, 
                      group['ST_GFP'].values, 
                      group['SpT_anti'].values)
      
    datasets[construct] = data
        

print(param_labels)
print(param_dict)
print(datasets)

x0 = [0.0, 0.0, 1.0] + n_push * [2.0]
bounds = [(0.0, 1.0), (None, None), (None, None)] + n_push * [(None, None)]

res = solve(datasets, phospho_noise, model_dict, param_dict, x0, bounds, verbose=True)


In [ ]:
model_dict = {'kinase_dead': ('background', 'background'),
              'sub_only': ('push', 'EE(L)*'),
              'u005': ('push', 'EE(L)*'), 
              'u030': ('push', 'EE(L)*'), 
              'u060': ('push', 'EE(L)*'),
              'u100': ('push', 'EE(L)*'),
              'EE(I)': ('push', 'EE(I)'), 
              'EE(L)': ('push', 'EE(L)'), 
              'EE(S)': ('push', 'EE(S)'), 
              'EE(E)': ('push', 'EE(E)'),
              'RR': ('push', 'RR')}




df_list = []
df = pd.read_csv("../data/push/u5.csv")
df['dataset'] = 'u005'
df_list.append(df)
df = pd.read_csv("../data/push/u30.csv")
df['dataset'] = 'u030'
df_list.append(df)
df = pd.read_csv("../data/push/u60.csv")
df['dataset'] = 'u060'
df_list.append(df)
df = pd.read_csv("../data/push/u100.csv")
df['dataset'] = 'u100'
df_list.append(df)
df = pd.read_csv("../data/push/Substrate only.csv")
df['dataset'] = 'sub_only'
df_list.append(df)
df = pd.read_csv("../data/push/Kinase Dead.csv")
df['dataset'] = 'kinase_dead'
df_list.append(df)


df = pd.read_csv("../data/push/EE(I).csv")
df['dataset'] = 'EE(I)'
df_list.append(df)
df = pd.read_csv("../data/push/EE(L).csv")
df['dataset'] = 'EE(L)'
df_list.append(df)
df = pd.read_csv("../data/push/EE(S).csv")
df['dataset'] = 'EE(S)'
df_list.append(df)
df = pd.read_csv("../data/push/EE(E).csv")
df['dataset'] = 'EE(E)'
df_list.append(df)
df = pd.read_csv("../data/push/RR.csv")
df['dataset'] = 'RR'
df_list.append(df)


df = pd.concat(df_list)


df = df[(df[df.columns[:-1]] >= 0).all(axis=1)].rename(columns={'Kinase': 'WT_anti', 'Substrate': 'ST_anti', 'Phosphorylation': 'SpT_anti'})


df = df.sample(frac=1.0, replace=False, random_state=776)

df.reset_index(inplace=True, drop=True)

df['SpT_anti/ST_anti'] = df['SpT_anti'] / df['ST_anti']


print(len(df.index), "/", len(df.index))

display(df)


ndatasets = df.groupby("dataset").ngroups
fig, axes = plt.subplots(ndatasets, 1, figsize=(4, 4*ndatasets),
                        sharex=True, sharey=True)

for i, (dataset, group) in enumerate(df.groupby("dataset")):
    
    sns.histplot(group, x='WT_anti', y='SpT_anti/ST_anti', 
                 log_scale=(True, True), ax=axes[i])
    
    axes[i].hlines(1e0, xmin=1e0, xmax=1e5, color='k', linestyle='--')
    axes[i].set_title(dataset)

plt.show()


In [ ]:
nbins_anti = 100
nbins_gfp = 100

writer_noise = noise.EmpNoiseModel("../data/Kinase Noise.csv", 
                                   'Flag Antibody', 'GFP - Area', 
                                   nbins_anti=nbins_anti, nbins_gfp=nbins_gfp, 
                                   verbose=True)
writer_noise.plot()

for dataset, group in df.groupby("dataset"):
    
    print(dataset)
    
    df.loc[group.index, 'WT_GFP'] = writer_noise.anti_to_GFP(group['WT_anti'], 
                                                             plot=False)
    
    
substrate_noise = noise.EmpNoiseModel("../data/Substrate Noise.csv", 
                                   'Myc Antibody', 'GFP - Area', 
                                   nbins_anti=nbins_anti, nbins_gfp=nbins_gfp, 
                                   verbose=True)
substrate_noise.plot()

for dataset, group in df.groupby("dataset"):
    
    print(dataset)
    
    df.loc[group.index, 'ST_GFP'] = substrate_noise.anti_to_GFP(group['ST_anti'], 
                                                                plot=False)



print(len(df))
df.dropna(inplace=True)
print(len(df))

display(df)

In [ ]:
def solve(models, model_dict, param_dict, x0, bounds, verbose=False):

    if verbose:
        start = time.time()

    def loss(x, args):
        

        Sigma2 = x[0]
        A = x[1]
        logvbgp = x[2]
        logvWSp = x[3]
        

        (models) = args

        loss = 0.0
        norm = 0.0
        for dataset in models:
            
            (model_type, model_id) = model_dict[dataset]
            model = models[dataset]
            
            N_data = model.N_data
            
            norm += N_data
                        
            if model_type == 'push':
                  
                Sigma2, A, logvbgp, logvWSp, logalphaWS = np.array(x)[param_dict[model_id]]

                loss += N_data * model.loss(
                    np.array([10**logalphaWS, 10**logvWSp, 10**logvbgp]), 
                    np.array([Sigma2, A, 0.0]))
            elif model_type == 'background':
                
                Sigma2, A, logvbgp = np.array(x)[param_dict[model_id]]
                
                loss += N_data * model.loss(
                    np.array([10**logvbgp]), 
                    np.array([Sigma2, A, 0.0]))
        
        loss /= norm
        
        return loss


    if verbose:
        print("Initial Loss:", loss(x0, (models)))


    res = opt.minimize(loss, x0, args=(models,), method='L-BFGS-B', 
                       jac='2-point', bounds=bounds, 
                       options={'iprint':101, 'eps': 1e-8, 
                                'gtol': 1e-8, 'ftol':1e-12})
    

    if verbose:
        print("Final Loss:", res.fun)

        end = time.time()

        print("Time Elapsed", end-start, "seconds")

        print(res)
        
    
    return res




In [ ]:
models = {}
param_dict = {}

n_push = 0

param_labels = [r"$\Sigma^2$", r"$A$", r"$\log_{10}(v_{bg}^p)$", r"$\log_{10}(v_{WS}^p)$"]

for dataset, group in df.groupby("dataset"):
    
    (model_type, model_id) = model_dict[dataset]
            
    if model_id not in param_dict:
        if model_type == 'background':
            param_dict[model_id] = [0, 1, 2]
        elif model_type == 'push':
            param_dict[model_id] = [0, 1, 2, 3, 4+n_push]
            n_push += 1
            
            param_labels.append(model_id + ": " + r"$\log_{10}(\alpha_{WS})$")
        
    
    if model_type == 'background':
        model = ppamp.Background()
        model.set_data(group['ST_GFP'].values.copy(), 
                       group['SpT_anti'].values.copy())

    elif model_type == 'push':
        model = ppamp.Push()
        model.set_data(group['WT_GFP'].values.copy(),
                       group['ST_GFP'].values.copy(), 
                       group['SpT_anti'].values.copy())
                
    models[dataset] = model

print(param_dict)

x0 = [0.05, 1.0, 0.0, 1.0] + n_push * [3.0]
bounds = [(1e-2, None), (1e-2, None), (None, None), (None, None)] + n_push * [(None, None)]

res = solve(models, model_dict, param_dict, x0, bounds, verbose=True)


In [ ]:
hess = la.inv(res.hess_inv.todense())

print("Noise parameters")
for i in range(2):
    display(Markdown(param_labels[i] + " = " + str(res.x[i])))

print("Model parameters:")
for i in range(2, len(res.x)):
    display(Markdown(param_labels[i] + " = " + str(res.x[i])))
    

s_list = []

for i, labeli in enumerate(param_labels):
    for j, labelj in enumerate(param_labels):
        s_list.append([labeli, labelj, np.log10(np.abs(hess[i, j]))])

df_hess = pd.DataFrame(s_list, columns=['param1', 'param2', 'hess'])
sns.heatmap(df_hess.pivot("param1", "param2", "hess"), 
            cbar_kws={'label': r"$\log_{10}(|H_{ij}|)$"}, 
            cmap='cividis', center=0)

plt.show()

evals, evecs = la.eigh(hess)

evals = evals[::-1]
evecs = evecs[:, ::-1]

s_list = []
for i, labeli in enumerate(param_labels):
    for j in range(len(evals)):
        s_list.append([labeli, "{:07.4f}".format(evals[j]), np.abs(evecs[i, j])])

df_evecs = pd.DataFrame(s_list, columns=['param', "PC (eigenval)", 'val'])
sns.heatmap(df_evecs.pivot("PC (eigenval)", "param", "val"), 
            cbar_kws={'label': "weight"}, 
            cmap='RdBu', center=0)

plt.show()

In [ ]:
hess_inv = res.hess_inv.todense()

zippers = {}


for dataset in model_dict:
    
    (model_type, model_id) = model_dict[dataset]
    
    if model_id not in param_dict:
        continue

    if model_type == 'push':
        
        if model_id not in zippers:
            idx = param_dict[model_id][4]
            zippers[model_id] = (res.x[idx], 0.01*np.sqrt(hess_inv[idx, idx]))


print(zippers)
            
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

y = 10**np.array([zippers[key][0] for key in zippers])
y_low =  y - 10**np.array([zippers[key][0]-zippers[key][1] for key in zippers])
y_up =  10**np.array([zippers[key][0]+zippers[key][1] for key in zippers]) - y

ax.bar(zippers.keys(), y, yerr=(y_low, y_up))

ax.set_yscale('log')
ax.set_xlabel("Zipper")
ax.set_ylabel("Inverse Binding Strength\n" + r"$\alpha_{WS} = k^{off}/k^{on}$ [GFP]")
plt.xticks(rotation=45)

plt.show()

In [ ]:


for dataset in models:
    model = models[dataset]
    (model_type, model_id) = model_dict[dataset]
    
    if isinstance(model, ppamp.Push):
        Sigma2, A, logvbgp, logvWSp, logalphaWS = np.array(res.x)[param_dict[model_id]]
        
        SpT_GFP_predict = model.predict_all(
            np.array([10**logalphaWS, 10**logvWSp, 10**logvbgp]))
    elif isinstance(model, ppamp.Background):
        Sigma2, A, logvbgp = np.array(res.x)[param_dict[model_id]]
        
        SpT_GFP_predict = model.predict_all(
            np.array([10**logvbgp]))
        
    df.loc[df[df.dataset==dataset].index, 'SpT_GFP_predict'] = SpT_GFP_predict
    
    
ST_noise = noise.LogNormNoiseModel(Sigma2=Sigma2, A=A, B=0.0)

df['SpT_anti_predict'] = ST_noise.GFP_to_anti(df['SpT_GFP_predict'])

df['SpT_GFP_predict/ST_GFP'] = df['SpT_GFP_predict'] / df['ST_GFP']
df['SpT_anti_predict/ST_anti'] = df['SpT_anti_predict'] / df['ST_anti']

display(df)

In [ ]:
ndatasets = df.groupby("dataset").ngroups
fig, axes = plt.subplots(ndatasets, 2, figsize=(8, 4*ndatasets),
                        sharex=True, sharey=True)

for i, (dataset, group) in enumerate(df.groupby("dataset")):
    
    sns.histplot(group, x='WT_anti', y='SpT_anti/ST_anti', 
                 log_scale=(True, True), ax=axes[i, 0])
    sns.histplot(group, x='WT_anti', y='SpT_anti_predict/ST_anti', 
                 log_scale=(True, True), ax=axes[i, 1])
    
    axes[i, 0].hlines(1e0, xmin=1e0, xmax=1e5, color='k', linestyle='--')
    axes[i, 1].hlines(1e0, xmin=1e0, xmax=1e5, color='k', linestyle='--')
    axes[i, 0].set_title("{0:} {1:}".format(dataset, "exp"))
    axes[i, 1].set_title("{0:} {1:}".format(dataset, "theory"))
    
    model = models[dataset]
    
    

plt.show()

In [ ]:
ndatasets = df.groupby("dataset").ngroups
fig, axes = plt.subplots(ndatasets, 1, figsize=(4, 4*ndatasets),
                        sharex=True, sharey=True)

for i, (dataset, group) in enumerate(df.groupby("dataset")):
    
    sns.histplot(group, x='WT_GFP', y='SpT_GFP_predict/ST_GFP', 
                 log_scale=(True, False), ax=axes[i], bins=32)
    
    axes[i].set_title("{0:} {1:}".format(dataset, "theoretical GFP"))
#     axes[i].set_ylim(0, 1)


#     ST_list = [1e0, 1e2, 1e4, 1e6]

 
#     palette = iter(sns.color_palette("Reds", len(ST_list)))
#     for ST in ST_list:
#         WT = np.logspace(0.0, 5.0, 100)
        
#         if dataset == 'kinase_dead':
            
#             model.set_data(ST*np.ones_like(WT), np.array([]))
#             SpT_predict = model.predict_all(
#                 np.array([10**logvbgp]))
            
#             color = next(palette)
#             axes[i, 0].plot(WT, SpT_predict/ST, '-', 
#                             color=color, lw=1.5)
#             axes[i, 1].plot(WT, SpT_predict/ST, '-', 
#                             color=color, lw=1.5)
            
#         else:
#             model.set_data(WT, ST*np.ones_like(WT), np.array([]))
#             SpT_predict = model.predict_all(
#                 np.array([10**logalphaWS, 10**logvWSp, 10**logvbgp]))
            
#             color = next(palette)
#             axes[i, 0].plot(WT, SpT_predict/ST, '-', 
#                             color=color, lw=1.5)
#             axes[i, 1].plot(WT, SpT_predict/ST, '-', 
#                             color=color, lw=1.5)

plt.show()

In [ ]:
ndatasets = df.groupby("dataset").ngroups
fig, axes = plt.subplots(ndatasets, 2, figsize=(8, 4*ndatasets),
                        sharex=True, sharey=True)

for i, (dataset, group) in enumerate(df.groupby("dataset")):
    
    sns.histplot(group, x='SpT_anti', 
                 log_scale=True, ax=axes[i, 0], label="exp", bins=64,
                 element="step", fill=False, stat='density')
    sns.histplot(group, x='SpT_anti_predict', 
                 log_scale=True, ax=axes[i, 0], label="theory", color='g', bins=64,
                 element="step", fill=False, stat='density')
    
    axes[i, 0].set_title(dataset)
    axes[i, 0].legend()
    
    sns.histplot(group, x='SpT_anti/ST_anti', 
                 log_scale=True, ax=axes[i, 1], label="exp", bins=64,
                 element="step", fill=False, stat='density')
    sns.histplot(group, x='SpT_anti_predict/ST_anti', 
                 log_scale=True, ax=axes[i, 1], label="theory", color='g', bins=64,
                 element="step", fill=False, stat='density')
    
    axes[i, 1].set_title(dataset)
    axes[i, 1].legend()

plt.show()